# Siamese U-Net Quickstart

**IMPORTANT**: Two packages packages need to be installed manually before running bio-image-unet: CUDA and PyTorch. To install CUDA 11.1 which is officially supported by PyTorch, navigate to [its installation page](https://developer.nvidia.com/cuda-11.1.1-download-archive) and follow the instructions onscreen. Because PyTorch depends on your CUDA installation version, it will need to be installed manually as well, through [the official PyTorch website](https://pytorch.org/get-started/locally/). Select the correct distribution of CUDA on this webpage and run the command in your terminal. bio-image-unet doesn't depend on a specific version of CUDA and has been tested with PyTorch 1.7.0+.

## 1. Introduction

The Siamese U-Net is an improvement on the original U-Net architecture. It adds an additional additional encoder that encodes an additional frame other than the frame that we are trying to predict, and uses a cross correlation layer to make the best inference for the data, detailed in [this paper](https://pubmed.ncbi.nlm.nih.gov/31927473/). This repository contains an implementation of this network.

This document serves both as a "Getting Started" tutorial and "Best Practices" documentation. If you need help using a class (one that is directly under the `biu.siam_unet` directory), the examples in this notebook should be helpful. If you need help using a function or if you are interested in seeing the full list of parameters for a certain function, you can run `help(whichever_interesting_function)` in the interactive shell or take a peek at the source code. 

Finally, to import the Siamese U-Net package, write `import biu.siam_unet as unet`.

## 2. Data preparation

Because Siam UNet requires an additional input for training, we need to utilize an additional frame and use the appropriate dataloader for that. For the purpose of this notebook, I will call the frame which we are trying to infer "current frame", and the frame which is before the current frame the "previous frame." 

#### If your input image is not a movie

In [ ]:
from bio_image_unet.siam_unet.helpers.generate_siam_unet_input_imgs import generate_coupled_image_from_self
from pathlib import Path
import os

# specify where the training data for vanilla u-net is located
training_data_loc = '/home/longyuxi/Documents/mount/deeptissue_training/training_data/amnioserosa/yokogawa/image'
training_data_loc = Path(training_data_loc)

# create a separate folder for storing Siam-UNet input images
siam_training_data_loc = training_data_loc.parent / "siam_image"
siam_training_data_loc.mkdir(exist_ok=True)

### multiprocessing accelerated, equivalent to 
## for img in training_data_loc.glob('*.tif'):
##     generate_coupled_image_from_self(str(img), str(siam_training_data_loc / img.name))

import multiprocessing

imglist = training_data_loc.glob('*.tif')
def handle_image(img):
    generate_coupled_image_from_self(str(img), str(siam_training_data_loc / img.name))

p = multiprocessing.Pool(10)
_ = p.map(handle_image, imglist)


In [ ]:
import tifffile
a = tifffile.imread('/home/longyuxi/Documents/mount/deeptissue_training/training_data/leading_edge/eCad/image/00.tif')
print(a.shape)

generate_coupled_image_from_self('/home/longyuxi/Documents/mount/deeptissue_training/training_data/leading_edge/eCad/image/00.tif', 'temp.tif')

#### If you know which frame you drew the label with

The dataloader in `siam_unet_cosh` takes an image that results from concatenating the previous frame with the current frame. If you already know which frame of which movie you want to train on, you can create this concatenated data using `generate_siam_unet_input_imgs.py`.

In [ ]:
movie_dir = '/media/longyuxi/H is for HUGE/docmount backup/unet_pytorch/training_data/test_data/new_microscope/21B11-shgGFP-kin-18-bro4.tif' # change this
frame = 10 # change this
out_dir = './training_data/training_data/yokogawa/siam_data/image/' # change this



from bio_image_unet.siam_unet.helpers.generate_siam_unet_input_imgs import generate_coupled_image

generate_coupled_image(movie_dir, frame, out_dir)

#### If you don't know which frame you drew the label with

If you have frames and labels, but you don't know which frame of which movie each frame comes from, you can use  `find_frame_of_image`. This function takes your query and compares it against a list of tif files you specify through the parameter `search_space`.

In [ ]:
image_name = f'./training_data/training_data/yokogawa/lateral_epidermis/image/83.tif'

razer_local_search_dir = '/media/longyuxi/H is for HUGE/docmount backup/all_movies'
tifs_names = ['21B11-shgGFP-kin-18-bro4', '21B25_shgGFP_kin_1_Pos0', '21C04_shgGFP_kin_2_Pos4', '21C26_shgGFP_Pos12', '21D16_shgGFPkin_Pos7']
search_space = [razer_local_search_dir + '/' + t + '.tif' for t in tifs_names]

from bio_image_unet.siam_unet.helpers.find_frame_of_image import find_frame_of_image

find_frame_of_image(image_name, search_space=search_space)


This function not only outputs what it finds to stdout, but also creates a machine readable output, location of which specified by `machine_readable_output_filename`, about which frames it is highly confident with at locating (i.e. an MSE of < 1000 and matching frame numbers). This output can further be used by `generate_siam_unet_input_images.py`.

In [ ]:
from bio_image_unet.siam_unet.helpers.generate_siam_unet_input_imgs import utilize_search_result

utilize_search_result(f'./training_data/training_data/yokogawa/amnioserosa/search_result_mr.txt', f'./training_data/test_data/new_microscope', f'./training_data/training_data/yokogawa/amnioserosa/label/', f'./training_data/training_data/yokogawa/siam_amnioserosa_sanitize_test/')


Finally, organize the labels and images in a way similar to this shown. An example can be found at `training_data/lateral_epidermis/yokogawa_siam-u-net`

```
training_data/lateral_epidermis/yokogawa_siam-u-net
|
├── image
│   ├── 105.tif
│   ├── 111.tif
│   ├── 120.tif
│   ├── 121.tif
│   ├── 1.tif
│   ├── 2.tif
│   ├── 3.tif
│   ├── 5.tif
│   ├── 7.tif
│   └── 83.tif
└── label
    ├── 105.tif
    ├── 111.tif
    ├── 120.tif
    ├── 121.tif
    ├── 1.tif
    ├── 2.tif
    ├── 3.tif
    ├── 5.tif
    ├── 7.tif
    └── 83.tif
```

## 3. Training

Training is simple. For example:

Note that the *invert* flag must be set to True for DeepTissue to segment the cells correctly (that is, the prediction would have white cell boundaries and black background).

In [ ]:
from bio_image_unet.siam_unet import *

dataset = 'amnioserosa/old_scope'
base_dir = '/home/longyuxi/Documents/mount/deeptissue_training/training_data/'

# path to training data (images and labels with identical names in separate folders)
dir_images = f'{base_dir}/{dataset}/siam_image/'
dir_masks = f'{base_dir}/{dataset}/label/'

print('starting to create training dataset')
print(f'dir_images: {dir_images}')
print(f'dir_masks: {dir_masks}')
# create training data set
data = DataProcess([dir_images, dir_masks], data_path='../delete_this_data', dilate_mask=0, aug_factor=10, create=True, clip_threshold=(0.2, 99.8), dim_out=(256, 256), shiftscalerotate=(0, 0, 0))

save_dir = f'/home/longyuxi/Documents/mount/trained_networks_new_siam/siam/{dataset}'
# create trainer
training = Trainer(data ,num_epochs=500 ,batch_size=12, load_weights=False, lr=0.0001, n_filter=32, save_iter=False, save_dir=save_dir, loss_function='logcoshTversky', loss_params=(1.5, 0.6))


training.start()


Note here that the value of the `n_filter` parameter is set to `32`. This is the depth of the first convolution layer in the U-Net architecture. Assignment of this value should depend on the amount of RAM that one's GPU has, but `32` layers has worked well for us and runs on a GTX 1080 with 8GB RAM. Should you change this value, you need to change the `n_filter` parameter when running `Predict` (next section) as well.

### A note on loss functions

One can switch the loss functions used to train the network. There are three options for the `loss_function` parameter: `BCEDice`, `Tversky` and `logcoshTversky`, and the two-element tuple `loss_params` to accompany each loss function.

#### BCE Dice Loss

The BCE Dice loss is a combination of the binary cross entropy loss (BCELoss) and Dice loss. Both are commonly used loss functions for image segmentation, and the BCE Dice loss is a weighted sum of them. The BCE Dice loss is defined as:

$$
\mathcal{L_\text{BCE Dice}} = \alpha * \mathcal{L_\text{BCE}} + \beta * \mathcal{L_\text{Dice}}
$$

We implemented $\mathcal{L_\text{BCE}}$, $\mathcal{L_\text{Dice}}$ and $\mathcal{L_\text{BCE Dice}}$ based on https://github.com/achaiah/pywick/blob/master/pywick/losses.py.

#### Tversky Loss

Tversky loss is defined as follows (as described in https://arxiv.org/pdf/1706.05721.pdf):

$$
\mathcal{L_\text{Tversky}} = \frac{\text{TP} + S}{\text{TP} + \alpha * \text{FP} + \beta * \text{FN} + S}
$$

Where TP, FP, and FN indicate true positives, false positives and false negatives respectively. $S$ is a smooth factor that is set to 1 by default and can't be changed in the current implementation. $\alpha$ and $\beta$ are weights on how much importance FP and FN should have, respectively. A higher $\alpha$ would make the network more sensitive on false positives, and are passed in through `loss_params`.

#### logcoshTversky Loss

As suggested by its name, the logcoshTversky loss is defined as 

$$
\mathcal{L_\text{Tversky}} = \log{(\cosh{(\mathcal{L_\text{Tversky}})})}
$$

And the parameters of `loss_params` are passed into $\alpha$ and $\beta$ of the inner Tversky loss.

#### Our experience with these loss functions:

`BCEDice` with `loss_params=(1.0, 1.0)` has consistently produced decent segmentation for us, so we think `BCEDice` can be used as a "default" loss function for running a dataset for the first time. `Tversky` and `logcoshTversky` losses, when trained with good $\alpha$ and $\beta$ parameters, can produce even better training results. The optimal weight of $\alpha$ and $\beta$ for the Tversky loss functions seems to depend on proportion of each image that is label vs background, since they alter the weight of false positives and false negatives.

## 4. Predict

Predicting is simple as well. Just swap in the parameters

In [ ]:
# load package
from bio_image_unet.siam_unet import *
import os
os.nice(10)
from  bio_image_unet.siam_unet.helpers import tif_to_mp4

base_dir = './'
out_dir = f'{base_dir}/predicted_out'
model = f'{base_dir}/models/siam_bce_amnio/model_epoch_100.pt'

tif_file = f'{base_dir}/training_data/test_data/new_microscope/21C04_shgGFP_kin_2_Pos4.tif'

result_file = f'{out_dir}/siam_bce_amnio_100_epochs_21C04_shgGFP_kin_2_Pos4.tif'
out_mp4_file = result_file[:-4] + '.mp4'

print('starting to predict file')
# predict file 
predict = Predict(tif_file, result_file, model, invert=False, resize_dim=(512, 512))
# convert to mp4
tif_to_mp4.convert_to_mp4(result_file, output_file=out_mp4_file, normalize_to_0_255=True)

Additionally, to evaluate the model's performance with different losses, one can also train the model across different models

In [ ]:
"""
For each image in the training dataset, run siam unet to predict.
"""

from pathlib import *

from bio_image_unet.siam_unet import *
import glob
import logging

def predict_all_training_data(image_folder_prefix, model_folder_prefix, model_loss_functions, datasets, output_directory):
    image_folder_prefix = Path(image_folder_prefix)
    model_folder_prefix = Path(model_folder_prefix)
    datasets = [Path(d) for d in datasets]
    output_directory = Path(output_directory)
    for dataset in datasets:
        for model_loss_function in model_loss_functions:
            try:
                current_model = Path(model_folder_prefix / model_loss_function / dataset / 'model.pt')
                for image in glob.glob((str) (image_folder_prefix / dataset) + "/image/*.tif"):
                    image_name = image.split('/')[-1]
                    result_name = Path(output_directory / dataset / Path(image_name[:-4] + '_' + model_loss_function + '.tif'))
                    _ = Predict(image, result_name, current_model, invert=False)
                    # _ = Predict(image, result_name, current_model, invert=False, resize_dim=None, n_filter=32)
            except:
                logging.error('{} in {} failed to execute'.format(model_loss_function, dataset))
                

if __name__ == '__main__':
    # BEGIN Full dataset
    folders = ["amnioserosa/yokogawa", "lateral_epidermis/40x", "lateral_epidermis/60x", "lateral_epidermis/yokogawa", "leading_edge/eCad", "leading_edge/myosin", "leading_edge/yokogawa_eCad", "nodes/old_scope", "nodes/yokogawa"]
    model_loss_functions = ['siam_bce_dice','siam_logcoshtversky', 'siam_tversky', 'siam_logcoshtversky_08_02', 'siam_logcoshtversky_15_06', 'siam_logcoshtversky_02_08', "siam_logcoshtversky_06_15", 'siam_tversky_08_02', 'siam_tversky_15_06']
    # END Full dataset

    # BEGIN Toy dataset
    # folders = ["lateral_epidermis/40x"]
    # model_loss_functions = ['siam_bce_dice','siam_logcoshtversky', 'siam_tversky', 'siam_logcoshtversky_08_02', 'siam_logcoshtversky_15_06']
    # END Toy dataset

    predict_all_training_data(image_folder_prefix='/home/longyuxi/Documents/mount/deeptissue_training/training_data', model_loss_functions=model_loss_functions, model_folder_prefix='/home/longyuxi/Documents/mount/trained_networks', datasets=folders, output_directory='/home/longyuxi/Documents/mount/deeptissue_test/output_new_shape')

# Appendix: An annotated structure of the siam_unet package

Below is an annotated structure of the siam_unet package. Use `help(function)` to read the docstring of each function for a better understanding.

```
Package                                         Use

.
├── __init__.py
├── data.py                                     dataloader script
├── siam_unet.py                                Siam U-Net model
├── train.py                                    training script
├── losses.py                                   loss functions
├── predict.py                                  prediction script
├── helpers                                     helper functions (usually not 
                                                        so useful except the 
                                                        ones mentioned in this notebook)
                                                        
│   ├── average_tifs.py                             averages a list of tiff files
│   ├── create_pixel_value_histogram.py             creates histograms for the 
                                                        pixel values in tif 
                                                        files. Useful for 
                                                        debugging during training
│   ├── cuda_test.py                                tests cuda functionality
│   ├── extract_frame_of_movie.py                   extract a certain frame of a 
                                                        tif movie 
│   ├── find_frame_of_image.py                      finds the frame number of 
                                                        a given query image 
                                                        within search_space.
│   ├── generate_plain_image.py                     generates a plain image
│   ├── generate_siam_unet_input_imgs.py            generates a coupled image 
                                                        for Siam U-Net training
│   ├── low_mem_tif_utils.py                        utilities for handling tif 
                                                        files with low memory 
                                                        usage
│   ├── threshold_images.py                         thresholds each frame of a 
                                                        tif movie
│   ├── tif_to_mp4.py                               uses ffmpeg to convert a tif 
                                                        movie to mp4
│   └── util.py                                     various utilities. see docstring

```